In [1]:
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sneakers_ml.features.features import get_train_val_test
from sneakers_ml.models.onnx import save_sklearn_onnx

In [2]:
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test("data/features/hog", "brands-classification")

In [5]:
param_grid = {"C": [0.1, 1, 10], "gamma": ["scale", "auto"], "kernel": ["linear"]}

svc = SVC()
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear']},
             scoring='accuracy', verbose=1)

In [7]:
pred = grid_search.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.7362534948741846

In [10]:
save_sklearn_onnx(grid_search.best_estimator_, X_train, "data/models/HOG-SVM-73acc.onnx")

In [11]:
param_grid = {"loss": ["log_loss", "hinge"], "alpha": [0.0001, 0.001]}

sgd = SGDClassifier()
grid_search = GridSearchCV(estimator=sgd, param_grid=param_grid, cv=5, scoring="accuracy", verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001],
                         'loss': ['log_loss', 'hinge']},
             scoring='accuracy', verbose=1)

In [14]:
pred = grid_search.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.7110904007455732

In [15]:
save_sklearn_onnx(grid_search.best_estimator_, X_train, "data/models/HOG-SGD-71acc.onnx")

In [3]:
model = CatBoostClassifier(verbose=True, iterations=200)

model.fit(X_train, y_train, eval_set=(X_val, y_val))

Learning rate set to 0.204354
0:	learn: 2.3539077	test: 2.3722972	best: 2.3722972 (0)	total: 2.49s	remaining: 8m 16s
1:	learn: 2.1945715	test: 2.2250850	best: 2.2250850 (1)	total: 4.97s	remaining: 8m 11s
2:	learn: 2.0953199	test: 2.1422196	best: 2.1422196 (2)	total: 7.46s	remaining: 8m 9s
3:	learn: 2.0154171	test: 2.0694040	best: 2.0694040 (3)	total: 9.95s	remaining: 8m 7s
4:	learn: 1.9536255	test: 2.0175847	best: 2.0175847 (4)	total: 12.5s	remaining: 8m 7s
5:	learn: 1.8779733	test: 1.9535358	best: 1.9535358 (5)	total: 15s	remaining: 8m 4s
6:	learn: 1.8302153	test: 1.9167333	best: 1.9167333 (6)	total: 17.4s	remaining: 8m
7:	learn: 1.7961325	test: 1.8918278	best: 1.8918278 (7)	total: 19.8s	remaining: 7m 56s
8:	learn: 1.7598785	test: 1.8692407	best: 1.8692407 (8)	total: 22.2s	remaining: 7m 51s
9:	learn: 1.7131081	test: 1.8364811	best: 1.8364811 (9)	total: 24.7s	remaining: 7m 49s
10:	learn: 1.6875890	test: 1.8230021	best: 1.8230021 (10)	total: 27.2s	remaining: 7m 47s
11:	learn: 1.6593768	

In [8]:
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.6486486486486487

In [9]:
model.save_model(
    "data/models/HOG-CatBoost-64acc.onnx",
    format="onnx",
    export_parameters={
        "onnx_domain": "ai.catboost",
        "onnx_model_version": 1,
        "onnx_doc_string": "iterations=100 default model",
        "onnx_graph_name": "CatBoostModel_for_MultiClassification",
    },
)